In [1]:
from os import link
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from urllib.parse import urljoin


import datetime
from pandas.core.common import random_state
from unidecode import unidecode

from itertools import zip_longest
import os
import glob
import datetime

import pprint 
import json
from pprint import pprint

In [2]:
def olx_scraping():
    def all_link(url):
      pNumber = 1
      all=[]
      while True:
        headers =  {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
        page = requests.get(url+ "?page=" + str(pNumber) , headers=headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        all_links = soup.find_all('a', class_='ads__item__ad--title')
        
        for i in all_links:
            all.append(i.attrs['href'])
        if soup.find("span",{"class":"item fright"}) ==None:
          break
        else:
            pNumber+=1
      return all
    list_of_dictt = []  
    def appartment_data(url,location,Type,r_s):
      headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
      list_of_links = all_link(url)
      for link in list_of_links:
        appartment_dict ={}
        result = requests.get(link,headers=headers)
        soup = BeautifulSoup(result.content,"lxml")
        table = soup.find_all("table",class_="item")
        for i in table:
          th = i.find('th').text.replace("\t","").replace("\n"," ").strip()
          strong = i.find('strong').text.replace("\t","").replace("\n"," ").strip()
          appartment_dict[th] = strong
        try:
          row = soup.find('ul' , {'class':'clearfix'}).find_all('li')
          appartment_dict['unit'] = row[2].text.replace("\t","").replace("\n"," ")
        except:
          appartment_dict['unit'] =None
        try:
          appartment_dict["price"]=soup.find("div",id="offerbox").find("strong").text.replace("\t"," ").replace("\r"," ").replace("\n"," ")
        except:
          appartment_dict["price"]=None
        try:
          appartment_dict["description"]=soup.find("p",{"class":"pding10 lheight20 large"}).text.replace("\t","").replace("\r"," ").replace("\n","")
        except:
          appartment_dict["description"]=None
          
        appartment_dict["city"]=location
        try:
          latitude = soup.findAll("div", class_="bgfff hidden br-1 vtop mapcontainer")
          data = [t.get('data-lat') for t in latitude if t.get('data-lat')] 
          appartment_dict["lat"]=None
        except:
          appartment_dict["lat"]=None
        try:
          longitude = soup.findAll("div", class_="bgfff hidden br-1 vtop mapcontainer")
          data = [t.get('data-lon') for t in longitude if t.get('data-lon')] 
          appartment_dict["lon"]=None
        except:
          appartment_dict["lon"]=None
        try:   
          appartment_dict["location1"]=soup.find("strong",{"class":"c2b small"}).string.replace("\t"," ").replace("\n"," ")
        except:
          appartment_dict["location1"]=None
        try:   
          appartment_dict["location2 "]=appartment_dict["location1"]
        except:
          appartment_dict["location2"]=None
        appartment_dict["type"]= Type
        appartment_dict["rent_sale"]=r_s
        try:
          appartment_dict["web_name"]=soup.find("a",id="headerLogo").text.strip()
        except:
          appartment_dict["web_name"]=None
        appartment_dict["link"]= link
        list_of_dictt.append(appartment_dict)
      return (list_of_dictt) 
    location = 'cairo'
    types=['apartments-duplex-for','villas-for','commercial-for','buildings-lands-other']
    for Type in types:
        if Type=='buildings-lands-other':
            rent_or_sale = ['1','2']
            for r_s in rent_or_sale:
                url = f"https://www.olx.com.eg/properties/{Type}/{location}/?search%5Bfilter_enum_sale_rent%5D%5B0%5D={r_s}"
                appartment_data(url,location,Type,r_s )
        else:
            rent_sale = ['sale','rent']
            for r_s in rent_sale:
                url = f"https://www.olx.com.eg/properties/{Type}-{r_s}/{location}/"
                appartment_data(url,location,Type,r_s ) 
  
    

  # #clean of data
 
    df = pd.DataFrame.from_dict(list_of_dictt)
    # df.to_csv('olx_Allcairo_scraped.csv', encoding='utf-8-sig', index=False)
    df.drop(columns=['الكماليات','الطابق','مقدم','كمبوند','مفروش','unit','شروط التسليم','تاريخ التسليم','طريقة الدفع'], inplace=True)

    df=df.rename(columns={'غرف نوم':'bedrooms','الحمامات':'bathrooms','المساحة (م٢)':'size','النوع':'unit'})

    df['price'] = df['price'].str.replace('ج.م', '').str.replace(',', '')
    # # change bathrooms arabic numbers
    df['bedrooms']=df['bedrooms'].replace('١٠+','10')
    df['bathrooms']=df['bathrooms'].replace('١٠+','10')
    df["bathrooms"] = df["bathrooms"].fillna('0').astype(int)
    df["bedrooms"] = df["bedrooms"].fillna('0').astype(int)
    df["bedrooms"] = df["bedrooms"].replace(0,np.nan)
    df["bathrooms"] = df["bathrooms"].replace(0,np.nan)

    df['size'] = pd.to_numeric(df['size'], errors='coerce')
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
    df = df.dropna(subset=['unit'])

    df['view'] = None
    df['source']='scraped'
    df['price_per_unit']=df['price']/df['size']
    df['price_per_unit']=df['price_per_unit'].round(2)
    df = df[df['location1'].str.contains('القاهرة')]
    column_names = ['link',
                    'bathrooms',
                    'bedrooms',
                    'city',
                    'location1',
                    'location2',
                    'price',
                    'size',
                    'price_per_unit',
                    'type',
                    'unit',
                    'lat',
                    'lon',
                    'view',
                    'description',
                    'web_name',
                    'source',
                    'rent_sale']
    df = df.reindex(columns=column_names)

    df['type']=df['type'].replace('apartments-duplex-for','سكنى').replace('villas-for','سكنى').replace('commercial-for','تجارى').replace('buildings-lands-other','أراضي')
    df['unit']=df['unit'].replace('فيلا منفصلة','فلل')
    df.loc[df['unit'] == 'محل', 'type'] = 'تجارى'
    df.loc[df['unit'] == 'مكتب', 'type'] = 'إدارى'
    df.loc[df['unit'] == 'مطعم و كافيه', 'type'] = 'تجارى'
    df.loc[df['unit'] == 'مصنع', 'type'] = 'تجارى'
    df.loc[df['unit'] == 'مخزن', 'type'] = 'تجارى'
    df.loc[df['unit'] == 'جراج', 'type'] = 'تجارى'
    df.loc[df['unit'] == 'عيادة', 'type'] = 'إدارى'
    df.loc[df['unit'] == 'مبنى تجارى كامل', 'type'] = 'تجارى'
    df['unit']=df['unit'].replace('سكنية','أرض مباني').replace('لكل الأغراض','أرض مباني').replace('زراعية','أرض زراعية').replace('تجارية','أرض تجارية')
    df.drop(df[df['unit'] == 'أخرى'].index, inplace = True)
    df['rent_sale']=df['rent_sale'].replace('rent','إيجار').replace('sale','بيع').replace('2','إيجار').replace('1','بيع')
    df['city']=df['city'].replace('cairo','القاهرة')
    df['web_name']=df['web_name'].replace('OLX Egypt','أوليكس')

    df['location1']=df['location1'].str.split('،').str[0]
    df['location1']=df['location1'].str.split(',').str[0]
    df['location2']=df['location1']

    df = df.dropna( how='any', subset=['price'])
    df = df.dropna( how='any', subset=['size'])
    df = df.dropna( how='any', subset=['location1'])
    df.to_csv('olx_Allcairo_clean.csv' ,index=False, encoding ='utf-8-sig')
olx_scraping()


<ipython-input-2-c01dcdeedd39>:99: FutureWarning: The default value of regex will change from True to False in a future version.
  df['price'] = df['price'].str.replace('ج.م', '').str.replace(',', '')


In [3]:
def propertyfinder_Scraping():    
    start = datetime.datetime.now()
    print("start ",start)
    
    location = []
    view = []
    rent = "ايجار"
    sale = "بيع"
    commercial = "تجاري"
    house = "سكني"
    Country = "مصر"
    category_Dict = {1: [sale, house] ,2: [rent, house],
                     3: [sale, commercial], 4: [rent, commercial]}
    dict_total = {}
    list_dict = []
    #category_num =2
    basic_url = "https://www.propertyfinder.eg"
    
    counter = 0
    links = []
    for category_num in (category_Dict.keys()):
        
        page_num = 0
        while (True):
            page_num += 1
            result = requests.get(
                f"https://www.propertyfinder.eg/ar/search?c={category_num}&dp=0&l=2254&ob=mr&page={page_num}")
            # save page content
            src = result.content
            # create "soup" object to parse content
            soup = BeautifulSoup(src, "lxml")
            # retreive html source code of the block
            ad_blocks = soup.find_all("div", {"class": "card-list__item"})
    
            # get the hyperlink carrying the whole information of the block (webpage)
            for i in range(len(ad_blocks)):
                l = basic_url+ad_blocks[i].find("a").attrs['href']
                links.append([l,category_num])
            next_page_link = soup.find(
                "a", {"class": "pagination__link pagination__link--next"})
#             if (page_num == 1 ):
            if(next_page_link == None):
                # print(page_num)
                break
    
        #json_path = "/content/drive/MyDrive/Colab Notebooks/All_Cairo_dataRent"
    
#     print(len(links))
    thousand = 1
    for link,cat_num in (links):
    
            result = requests.get(link)
            src = result.content
            soup = BeautifulSoup(src, "lxml")
            dict_total['link'] = link
    
            grand_parent = soup.find("div", {"class": "property-facts"})
        # Get the unit (نوع العقار)
            try:
                icon = (grand_parent.find(
                    "svg", {"class": "property-facts__icon", "viewbox": "0 0 17 18"}))
    
                dict_total['unit'] = (
                    icon.parent.next_sibling.text.replace("  ", "").replace("\n", ""))
            except:
                dict_total['unit'] = None
    
        # Get the area of the unit in "Meters" and cast it to integer
            try:
                icon = (grand_parent.find(
                    "svg", {"class": "property-facts__icon", "viewbox": "0 0 18 18"}))
                a_string = icon.parent.next_sibling.span.next_sibling.next_sibling.text
    
                dict_total['size'] = (int(a_string.split()[0].replace(",", "")))
            except:
                dict_total['size'] = None
    
        # Get the number of rooms and cast it to integer
            try:
                icon = (grand_parent.find(
                    "svg", {"class": "property-facts__icon", "viewbox": "0 0 19 15"}))
                dict_total['bedrooms'] = (
                    icon.parent.next_sibling.text.replace("\n", ""))
    
            except:
                dict_total['bedrooms'] = None
            try:
                icon = (grand_parent.find(
                    "svg", {"class": "property-facts__icon", "viewbox": "0 0 19 13"}))
                dict_total['bathrooms'] = (
                    icon.parent.next_sibling.text.replace(" ", ""))
            except:
                dict_total['bathrooms'] = None
    
        # Get the number of bathrooms , price and meter price
            try:
                dict_total['price'] = int(
                    soup.find("div", {"class": "property-price"}).text.split()[0].replace(",", ""))
                dict_total['price_per_unit'] = (
                    dict_total['price'] / dict_total['size'])
            except:
                dict_total['price'] = None
                dict_total['price_per_unit'] = None
    
        # Get the city,location1 and location2
            try:
                source = soup.find("div", {"class": "breadcrumb"})
                child = source.find_all(
                    "a", {"class": "text text--size1 link link--underline"})
                dict_total['city'] = (child[0].text)
                dict_total['location1'] = (child[1].text)
                if (len(child) > 2):
                    dict_total['location2'] = ((child[2].text))
    
                else:
                    dict_total['location2'] = (child[1].text)
            except:
                dict_total['location1'] = None
                dict_total['location2'] = None
    
            # تفاصيل و مزايا العقار
            try:
                dict_total['description'] = (soup.find(
                    "div", {"class": "property-page__description"}).div.text.replace("\n", ""))
            except:
                dict_total['description'] = None
    
            dict_total['web_name'] = "Property finder"
            dict_total['view'] = (None)
            dict_total['source'] = "scraped"
            #  "ايجار","بيع"
            dict_total['rent_sale'] = (category_Dict[cat_num][0])
            # "تجاري","سكني"
            dict_total['type'] = (category_Dict[cat_num][1])
            list_dict.append(dict_total.copy())
            counter += 1
            if (counter == thousand*1000) :
            #   print(counter)
              thousand+=1
    # print(counter)
    
    
    df = pd.DataFrame.from_dict(list_dict)
     #  df.to_csv('propertyFinder_Scraped_data.csv', index=False, header=True, encoding='utf-8-sig')
    
    # print("Scraped")
    
    
    '''***************************cleaning*****************************************'''
    
    df['cleaned_bathrooms'] = df['bathrooms'].str.replace("+", "")
    df.loc[df['cleaned_bathrooms'] == "غيرمتوفر", 'cleaned_bathrooms'] = np.nan
    df.loc[df['cleaned_bathrooms'] == "لايوجد", 'cleaned_bathrooms'] = np.nan
    
    df = df.astype({'cleaned_bathrooms': float})
    
    for i in range(len(df)):
        if(df.at[i, 'bedrooms'] == None):
            continue
        num = df.at[i, 'bedrooms'].split()
        if (num[0].replace("+", "").isdigit() == True):
            if (len(num) > 1):
                df.at[i, 'cleaned_bedrooms'] = float(num[0].replace("+", "")) + 1
            else:
                df.at[i, 'cleaned_bedrooms'] = float(num[0].replace("+", ""))
        else:
            string = df.at[i, 'bedrooms'].replace(" ", "")
            if ((string == "غيرمتوفر") or (string == "غيرمتوفر+غرفةخادمة") or (string == "لايوجد")):
                df.at[i, 'cleaned_bedrooms'] = np.nan
            else:
                if (string == "استوديو"):
                    df.at[i, 'cleaned_bedrooms'] = 1.0
                else:
                 # استوديو + غرفة خادمة
                    df.at[i, 'cleaned_bedrooms'] = 2.0
    
    df['web_name']=df['web_name'].replace('Property finder','بروبرتي فايندر')
    df['bedrooms'] = df['cleaned_bedrooms']
    df['bathrooms'] = df['cleaned_bathrooms']
    
    df.drop(['cleaned_bedrooms', 'cleaned_bathrooms'], axis=1, inplace=True)
    
    
    df['lon'] = np.nan
    df['lat'] = np.nan
    
    df = df.reindex(columns=['link', 'bathrooms', 'bedrooms', 'city', 'location1', 'location2', 'price',
                    'size', 'price_per_unit', 'type', 'unit','lat','lon' ,'view', 'description', 'web_name', 'source', 'rent_sale'])
    
    
    df.to_csv('propertyFinder_cleaned_data.csv', index=False, header=True, encoding='utf-8-sig')
        
    end = datetime.datetime.now()
    
    print("end ",end)
    
    print("duration ",end - start)

propertyfinder_Scraping()

start  2022-01-19 22:08:39.949179
100
end  2022-01-19 22:10:05.113749
duration  0:01:25.164570


<ipython-input-3-d34e7c14052f>:151: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['cleaned_bathrooms'] = df['bathrooms'].str.replace("+", "")


In [4]:
def TageerScrape():
    list_of_dict = []
    
    def scrape():  
        for j in kind:
#             print(j)
            pageNum = 1
            url_site = f"https://tageer.net/bhth.html?title=&category%5b0%5d={j}&location%5b0%5d=cairo&page={pageNum}"
            res_out = requests.get(url_site)
            src_out = res_out.content
            soup_out = BeautifulSoup(src_out, 'lxml')
            try:
                lastpage = soup_out.find('a', {'title': 'الأخير '}).attrs['href']
            except:
                lastpage = url_site
            
            while True: 
#                 print("page ", pageNum)
                url_site = f"https://tageer.net/bhth.html?title=&category%5b0%5d={j}&location%5b0%5d=cairo&page={pageNum}"
                pageNum += 1
                res = requests.get(url_site)
                src = res.content
                soup = BeautifulSoup(src, 'lxml')
                blocks = soup.find_all('a', {'class' : "big-txt"})
                x = 1
                for i in blocks:
#                     print("block :",x)
                    x += 1
                    dic = {}
                    url_in = i.attrs['href']
                    res_in = requests.get(url_in)
                    src_in = res_in.content 
                    soup_in = BeautifulSoup(src_in, 'lxml')
                    ti = i.text.strip()
                    try :
                        pri = soup_in.find('div', {'class': 'col-xs-12 col-sm-2 price price-curry'}).text.strip()
                    except:
                        pri = ""
                    try:
                        loc = soup_in.find('div', {'class': 'col-xs-12 col-sm-2'}).text.strip()
                    except:
                        loc = ""
                    try:
                        dis = soup_in.find('div', {'class': 'tab-pane fade active in'}).find('p').text.strip()
                    except:
                        dis = ""
                    try:
                        all_de = soup_in.find('table', {'class': 'table table-striped'}).find_all('tr')
                        for i in all_de:
                            key = i.find('td').text.strip()
                            value = i.find('td').findNext('td').text
                            dic[key] = value 
                    except:
                        print("No Table")

                    dic['link'] = url_in
                    dic['title'] = ti
                    dic['price'] = pri
                    dic['location1'] = loc
                    dic['description'] = dis
                    dic['type'] = j
                    list_of_dict.append(dic)
#                 break
                if url_site == lastpage:
                    break
            
#             print("Done",j)

    kind = ['flats-for-rent', 'shkk-k-non-kdym', 'furnished-flats', 'rooms-for-rent', 'villas-for-rent', 'business-rent-4',
             'shops-for-rent', 'offices-rent', 'store-land-rent', 'equipment-rent']
    
    scrape()
    print("Done Tageer")
    df = pd.DataFrame(list_of_dict)
    #df.to_csv(r'Tageer.csv', index = False, header = True)


    df.rename(columns= {'عدد الحمامات' : 'bathrooms',
                        'عدد الغرف' : 'bedrooms', 
                        'المساحة بالمتر' : 'size'
                       }, inplace = True,errors='raise')

    df.drop(['مفروش', 'الطابق', 'يضاف رسوم السمسرة','الأسانسير','السعر قابل للتفاوض', 'قيمة المقدم'], axis = 1, inplace = True)
    df.dropna(inplace = True)

    df['price'] = df['price'].str.replace('EGP',"")
    df['price'] = df['price'].str.replace(',',"")
    df.drop(df.index[df['price'] == ''], inplace = True)
    df.drop(df.index[df['size'] == ''], inplace = True)
    df.reset_index(inplace=True)
    df['price'] = df['price'].astype(float)
    df['size'] = df['size'].astype(float)
    del df['index']
    
    ####
    df['city'] = "القاهرة"
    df['location2'] = df['location1']
    df['price_per_unit'] = df['price'] / df['size']
    df['web_name'] = 'تأجير' 
    df['view'] = None
    df['source'] = 'scraped'
    df['rent_sale'] = 'ايجار'
    df['lat'] = None
    df['lon'] = None
    ###
    typeMaping = {'flats-for-rent' : 'شقق',
          'shkk-k-non-kdym' : 'شقق 59سنة',
          'furnished-flats' : 'شقق مفروشة',
          'rooms-for-rent' : 'غرف',
          'villas-for-rent' : 'فيلات و شاليهات',
          'business-rent-4' : 'مشروعات',
          'shops-for-rent' : 'محلات',
          'offices-rent' : 'مكاتب ومباني',
          'store-land-rent' : 'مخازن واراضي',
          'equipment-rent' : 'مصانع ومعدات'}
    newTyping = {
          'شقق' : 'سكني',
          'شقق 59سنة' : 'سكني',
          'شقق مفروشة' : 'سكني',
          'غرف' : 'سكني',
          'فيلات و شاليهات' : 'سكني',
          'مشروعات' : 'تجاري',
          'محلات' : 'تجاري',
          'مكاتب ومباني' : 'تجاري',
          'مخازن واراضي' : 'تجاري',
          'مصانع ومعدات' : 'تجاري'
    }
    df['bathrooms'] = df['bathrooms'].str.replace('+','')
    df['bedrooms'] = df['bedrooms'].str.replace('+','')
    df['bathrooms'] = pd.to_numeric(df.bathrooms.apply(unidecode), errors='coerce')
    df['bedrooms'] = pd.to_numeric(df.bedrooms.apply(unidecode), errors='coerce')
    
    df['bedrooms'] = df['bedrooms'].astype(int)
    df.drop(df[df['bedrooms'].isna()].index , inplace =True)
    df.drop(df[df['description'].isna()].index , inplace =True)
    
    
    df["unit"] = df['type'].map(typeMaping)
    df.drop(['type'], axis = 1, inplace = True)
    df['type'] = df['unit'].map(newTyping)
    #print(df.info())
    df.drop_duplicates()
    df = df.reindex(columns=['link',  'bathrooms',  'bedrooms', 
    'city', 'location1', 'location2', 'price', 'size', 'price_per_unit', 'type',  'unit', 
    'lat', 'lon', 'view', 'description', 'web_name', 'source', 'rent_sale'])
    df.to_csv(r'Tageer1.csv', index = False, header = True, encoding ='utf-8-sig')




TageerScrape()


flats-for-rent
page  1
block : 1
block : 2
block : 3
block : 4
block : 5
block : 6
block : 7
block : 8
block : 9
block : 10
block : 11
block : 12
block : 13
block : 14
block : 15
block : 16
block : 17
block : 18
block : 19
block : 20
Done flats-for-rent
shkk-k-non-kdym
page  1
block : 1
block : 2
block : 3
block : 4
block : 5
block : 6
block : 7
block : 8
block : 9
block : 10
block : 11
block : 12
block : 13
block : 14
block : 15
block : 16
block : 17
block : 18
block : 19
block : 20
Done shkk-k-non-kdym
furnished-flats
page  1
block : 1
block : 2
block : 3
block : 4
block : 5
block : 6
block : 7
block : 8
block : 9
block : 10
block : 11
block : 12
block : 13
block : 14
block : 15
block : 16
block : 17
block : 18
block : 19
block : 20
Done furnished-flats
rooms-for-rent
page  1
block : 1
block : 2
block : 3
block : 4
block : 5
block : 6
block : 7
block : 8
block : 9
block : 10
block : 11
block : 12
block : 13
block : 14
block : 15
block : 16
block : 17
block : 18
block : 19
block : 20


<ipython-input-4-2ef829eac283>:127: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['bathrooms'] = df['bathrooms'].str.replace('+','')
<ipython-input-4-2ef829eac283>:128: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['bedrooms'] = df['bedrooms'].str.replace('+','')


In [5]:
def open_souqscrapping():

  def next_page(headers, soup):
    next_page = soup.find('li', {'class':'next'}).find('span')['onclick'].replace("location.href='" , '').replace("'" , '')
    result = requests.get('https://eg.opensooq.com' + next_page)
    src=result.content
    soup = BeautifulSoup(src,"lxml")
    return soup

  def get_all_links(url):
  all_links = []
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
  result = requests.get(url, headers=headers)
  soup = BeautifulSoup(result.content, 'lxml')
  while soup:      
    links_in_page = soup.find_all('h2' , {'class':'fRight mb15'})
    links = ['https://eg.opensooq.com' + link.find('span')['data-href'] for link in links_in_page]
    # print(links)
    all_links.extend(links)
    try:
      soup = next_page(headers, soup)
    except Exception as e:
      print(e)
      break

  return all_links

  global df_list 
  df_list = []
  def scrap_data(loction, url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
    links_list = get_all_links(url)
    # print(links_list)
    for link in links_list:
      property_dict = {}
      try:
        result = requests.get(link, headers)
        soup = BeautifulSoup(result.content, 'lxml')
        table = soup.find_all('li', {'class':'inline vTop relative mb15'})
        for i in table:
          try:
            td1 = i.text.replace(i.find_next('span').text, '').strip()
          except:
            td1 = None
          try:
            td2 = i.find_next('span').text.strip()
          except:
            td2 = None
          try:
            property_dict[td1] = td2
          except:
            property_dict[td1] = None
        try:
          property_dict['link'] = link.strip()
        except:
          property_dict['link'] = None
        row = soup.find('ul',{'class':'breadcrumbs'}).find_all('li')
        try:
          property_dict['rent_sale'] = row[2].text
        except:
          property_dict['rent_sale'] = None
        try:
          property_dict['unit'] = row[3].text
        except:
          property_dict['unit'] = None
        try:
          property_dict['description'] = soup.find('div', {'class':'postDesc overflowHidden'}).find('p').text.strip().replace("\t","").replace("\r"," ").replace("\n","")
        except:
          property_dict['description'] = None
        try:
          property_dict['web_name'] = 'السوق المفتوح'
        except:
          property_dict['web_name'] = None
        property_dict_copy= property_dict.copy()
        df_list.append(property_dict_copy)
      except Exception as e:
        print(e)

  location = 'القاهرة'
  rent_or_sale =['عقارات-للبيع', 'عقارات-للايجار']
  for rent in rent_or_sale:
      url = f'https://eg.opensooq.com/ar/{location}/{rent}/all'
      scrap_data(location, url)

  df = pd.DataFrame.from_dict(df_list)

  # df.to_csv('cairo.csv', index=False, encoding='utf-8-sig')

  ##--------------------------------------------------------------------- CLEAN ----------------------------------------------------------------------------------------------##

  df = df.rename(columns={'المدينة :':'city', 'الحي:':'location1', 'عدد الغرف:':'bedrooms', 'عدد الحمامات:':'bathrooms', 'مساحة البناء:':'size','السعر:':'price', 'النوع:':'type'})
  # df.drop(['الطابق:', 'مفروشة/غير مفروشة:', 'عمر البناء:', 'مساحة الأرض:', 'عدد الطوابق:', 'مخصصة لـ:','مدة الإيجار:',  'طريقة الدفع:', 'الدولة:', 'نوع العقار:' ], axis=1, inplace=True)
  df.drop(['الطابق:', 'مفروشة/غير مفروشة:', 'عمر البناء:', 'مساحة الأرض:', 'عدد الطوابق:', 'مخصصة لـ:','مدة الإيجار:',  'طريقة الدفع:' ], axis=1, inplace=True)

  df['price'] = df['price'].str.replace('جنيه', '') 
  df['price'] = df['price'].str.replace(',', '')
  df['bathrooms'] = df['bathrooms'].str.replace('حمّامين', '٢') 
  df['bathrooms'] = df['bathrooms'].str.replace('حمّامات', '') 
  df['bathrooms'] = df['bathrooms'].str.replace('حمّام', '١') 
  df['bathrooms'] = df['bathrooms'].str.replace('+', '')
  df['bedrooms'] = df['bedrooms'].str.replace('غرف نوم', '')
  df['bedrooms'] = df['bedrooms'].str.replace('غرفة نوم', '1')
  df['bedrooms'] = df['bedrooms'].str.replace('غرفتا نوم', '')  
  df['bedrooms'] = df['bedrooms'].str.replace('غرف', '')
  df['bedrooms'] = df['bedrooms'].str.replace('ستوديو', '1')
  df['bedrooms']=df['bedrooms'].replace('١٠+','10')
  df['bathrooms']=df['bathrooms'].replace('١٠+','10')
  df["bathrooms"] = df["bathrooms"].fillna('0').astype(int)
  df["bedrooms"] = df["bedrooms"].fillna('0').astype(int)
  df["bedrooms"] = df["bedrooms"].replace(0,np.nan)
  df["bathrooms"] = df["bathrooms"].replace(0,np.nan)
  df['size'] = df['size'].str.replace('م٢', '')
  df['unit'] = df['unit'].str.replace('للبيع', '') 
  df['unit'] = df['unit'].str.replace('للايجار', '')
  df['rent_sale'] = df['rent_sale'].str.replace('للبيع', 'بيع') 
  df['rent_sale'] = df['rent_sale'].str.replace('للايجار', 'إيجار')
  df['unit'] = df['unit'].str.replace('\n', '')
  df['rent_sale'] = df['rent_sale'].str.replace('عقارات', '')

  df['location2'] = df['location1']

  df['size'] = pd.to_numeric(df['size'], errors='coerce') 
  df['price'] = pd.to_numeric(df['price'], errors='coerce') 
  df['price_per_unit'] = df['price'] / df['size']
  df['price_per_unit'] = df['price_per_unit'].round(2)

  column_names = ['link',     
                  'bathrooms',
                  'bedrooms',
                  'city',
                  'location1',
                  'location2',
                  'price',
                  'size',                     
                  'price_per_unit',
                  'type',
                  'unit',
                  'lat',
                  'lon',
                  'view',
                  'description',
                  'web_name',
                  'source', 
                  'rent_sale']
  df = df.reindex(columns=column_names)

  df['type'] = df['type'].str.replace('مكاتب - عيادات', 'إدارى')
  df['type'] = df['type'].str.replace('محل', 'تجارى') 
  df['type'] = df['type'].str.replace('مخزن/مستودع', 'تجارى') 
  df['type'] = df['type'].str.replace('مجمّع/مبنى تجاري', 'تجارى')
  df['type'] = df['type'].str.replace('معرض', 'تجارى') 
  df['type'] = df['type'].str.replace('مطعم - كوفي شوب', 'تجارى')
  df['type'] = df['type'].str.replace('فندق', 'سكنى') 
  df['type'] = df['type'].str.replace('طابق كامل', 'إدارى') 
  df['type'] = df['type'].str.replace('سوبر ماركت', 'تجارى') 
  df['type'] = df['type'].str.replace('سكن موظفين', 'سكنى') 
  df['type'] = df['type'].str.replace('فيلا تجارية', 'تجارى')

  df['type'].fillna('سكنى',inplace=True)

  df= df.dropna( how='any', subset=['price'])  
  df= df.dropna( how='any', subset=['size']) 
  df= df.dropna( how='any', subset=['location1'])
  df['source'] = 'Scraped'
  df['web_name'] = 'السوق المفتوح'
  df.drop(df[df['type'] == 'أخرى'].index, inplace = True)
  df.drop(df[df['location1'] == 'أخرى'].index, inplace = True)

  df = pd.DataFrame.from_dict(df, orient='columns')
  df.to_csv('opensooq.csv', index=False,  encoding ='utf-8-sig')


open_souqscrapping()

<ipython-input-5-533c6143dfe4>:93: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['bathrooms'] = df['bathrooms'].str.replace('+', '')


In [6]:
def semsar_function():
    
    gov_id = 984
    
    page_num = 1
    links = []
    titles = []
    descriptions = []
    unites = []
    locations = []
    dictionaries_list = []
    url = []
    while True :
        try:
            result = requests.get(f"https://www.semsarmasr.com/3akarat?title=%C7%DA%E1%C7%E4%content20%DA%DE%C7%D1%C7%CA%20%DD%ED%20%C7%E1%DE%C7%E5%D1%C9%20%E3%D5%D1&r=70&g=984&a=0&cid=0&p={page_num}")
            soup = BeautifulSoup(result.content , "lxml" )

            page = soup.find('div',{'id':'paging'}).find_all("a"  ) #,{"id" : "PageNum"}

            for i in range(len(page)):
                url.append(page[i].text)
                
        #    print(url)
       
        except:
            url.append('null')
        
        
        
        page_limit = url[-3]
    
        divs = soup.find_all('div' , {'class':'ListInfo'})
        #print(len(divs))
        for div in divs:
            link = div.find('a')['href']
            links.append(link)
    
        #print(page_num , page_limit)
    
        if page_num == int(page_limit):
            break
    
        page_num+=1

                           
    for link in links:
        result = requests.get(link)
        src = result.content
        soup = BeautifulSoup(src ,"lxml")
        table = soup.find_all('div' , {'id':'DetRrow'})
        try:
            title = soup.find("div" , {"class" : "PageTitlesStyle"})
        except: title='null'

        try:
            description = soup.find("p" , {"style" : "padding:5px;white-space:pre-line"})
        except: description = 'null'

        try:
            city = soup.find('div' , {'class':'PathStyle'}).find_all("span"  )[2]
        except: city = 'null'

        try:
            location1 = soup.find('div' , {'class':'PathStyle'}).find_all("span"  )[3]
        except: location1 = 'null'

        try:
            location2 = soup.find('div' , {'class':'PathStyle'}).find_all("span"  )[4]
        except: location2 = 'null'

        try:
            unit = soup.find('div' , {'class':'PathStyle'}).find_all("span"  )[-1]
        except: unit = 'null'

        #print("loc >>>" location.text)
        #print("title >>> "title.text)

        # print(titles)
        # print(descriptions)
        # print(locations)
        dic = {}
        for i in table:
            key = i.find('div')
            value = key.find_next('div')
            dic[key.text.strip()] = value.text.strip()

            dic["source"] = "scraped"
            dic["web_name"] = "سمسار"
            dic["lon"] = "None"
            dic["lat"] = "None"

        try:
            dic["title"] = title.text.strip()
        except: dic["title"] =  'null'

        try:
            dic['description'] = description.text.strip()
        except: dic['description'] =  'null'

        try:
            dic["location1"] = location1.text.strip()
        except: dic["location1"] =  'null'

        try:
            dic["city"] = city.text.strip()
        except: dic["city"] =  'null'

        try:
            dic["location2"] = location2.text.strip()
        except:  dic["location2"] =  'null'

        try:
            dic["unit"] = unit.text.strip()
        except:  dic["unit"] =  'null'

        try:
            dic["link"] = link
        except:  dic["link"] = 'null'

        #pprint(dic)

        #print(key.text, ' >>>>>>>> ' , value.text)
        #print(dic)
        dictionaries_list.append(dic) 

    df = pd.DataFrame (dictionaries_list)


# Cleaning the data to put it in determined format


    
    #df.head(n=10)
    df.rename({'السعر:': 'price', 'الغرض:':'rent_sale' , 'القسم:':'type' , 'المساحة:':'size' , 'عدد الغرف:':'bedrooms' , 'عدد الحمامات:':'bathrooms' }, axis=1, inplace=True)
    #df.head(n=2)
    df['view'] = ''
    df.head(n=2)
    #print(df.columns)
    df.drop(['رقم الدور:' , 'title' , 'نوع التشطيب:' ], axis=1, inplace=True)
    df.drop(['الفرش:' , 'رقم الإعلان:' ], axis=1, inplace=True)
    #print(df.columns)
    df.drop(['تاريخ البناء:' , 'عدد الأدوار:' , "طبيعة المُعلن:" ], axis=1, inplace=True)
    #df.head(n=5)
    df['price']=df['price'].str.replace(',','')
    #df.head(n=5)
    df['price'] = df.price.str.extract('(\d+)')
    df['price']
    df['unit']=df['unit'].str.extract('(^[^\s]+)')
    df['unit']
    #df.head(n=5)
    df['size']=df['size'].str.replace(',','')
    df = df.assign(size = lambda x: x['size'].str.extract('(\d+)'))
    df['size']
    #df.head(n=5)
    df['price'] = df['price'].astype(str)
    df['price'] = df['price'].astype(float)
    df['size'] = df['size'].astype(float)
    df["price_per_unit"] = df["price"].div(df["size"].values)
    df["location2"] = df["location1"]

    df.loc[df['type'] == 'عقارات سكنية', 'type'] = 'سكني'
    df.loc[df['type'] == 'أراضي زراعية ومزارع للبيع', 'type'] = 'اراضي'
    df.loc[df['type'] == 'عقارات تجارية', 'type'] = 'تجاري'
    df.loc[df['type'] == 'عقارات إدارية', 'type'] = 'اداري'
    df.loc[df['type'] == 'عقارات سياحية', 'type'] = 'تجاري'
    df.loc[df['type'] == 'عقارت طبية', 'type'] = 'تجاري'
    df.loc[df['type'] == 'عقارات صناعية', 'type'] = 'تجاري'
    df.loc[df['type'] == 'عقارات متنوعة', 'type'] = 'تجاري'
    df.loc[df['type'] == 'عقارات تعليمية', 'type'] = 'تجاري'
    df['rent_sale'] = df['rent_sale'].str.replace('للبيع', 'بيع') 
    df['rent_sale'] = df['rent_sale'].str.replace('للايجار', 'إيجار')
    #df.head(n=5)
    #print(df.columns)
    df = df.reindex(columns=['link', 'bathrooms', 'bedrooms', 'city', 'location1', 'location2', 'price',
                             'size', 'price_per_unit', 'type', 'unit','lat','lon' ,'view', 'description', 'web_name', 'source', 'rent_sale'])
    df.head(n=5)
    df.to_csv('semsar_data.csv', index=False, header=True, encoding='utf-8-sig')

# Calling the Function

semsar_function()


In [9]:
df1= pd.read_csv('Tageer1.csv')
df2= pd.read_csv('semsar_data.csv')
df3= pd.read_csv('olx_Allcairo_clean.csv')
df4= pd.read_csv('opensooq.csv')
df5= pd.read_csv('propertyFinder_cleaned_data.csv')
#concat
df = pd.concat([df1, df2,  df3, df4, df5])

df.dropna(subset = ["location1"], inplace=True)
df.dropna(subset = ["location2"], inplace=True)
df.dropna(subset = ["price"], inplace=True)
df.dropna(subset = ["size"], inplace=True)
df.drop(df[df['location1'] == 'عقارات سكنية في القاهرة مصر'].index,inplace = True)
df.drop_duplicates(inplace=True)
df.reset_index(inplace = True, drop = True)
df['location1']=df['location1'].str.split('،').str[0]
df['location1']=df['location1'].str.split(',').str[0]
df['location2']=df['location2'].str.split('،').str[0]
df['location2']=df['location2'].str.split(',').str[0]



df.to_csv('myAllData.csv', index=False, header=True, encoding='utf-8-sig')